In [31]:
# instal requirements
import os
import sys

requirements_file = 'requirements.txt'

if not os.path.exists(requirements_file):
    print(f"{requirements_file} not found. Please create it with the required packages.")
    sys.exit(1)

!{sys.executable} -m pip install -r requirements.txt


In [ ]:
with open("openai_api.key", "r") as file:
    openai_api_key = file.read().strip()

if not openai_api_key:
    print("OpenAI API key not found.")
    sys.exit(1)
else:
    print("OpenAI API key found.")
    # set the environment variable
    os.environ["OPENAI_API_KEY"] = openai_api_key

from langchain_openai import ChatOpenAI

with open("openai_api.key", "r") as file:
    openai_api_key = file.read().strip()


OpenAI API key found.


In [41]:
import yfinance as yf

def get_stock_price(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        price = stock.history(period="1d")["Close"][-1]
        return f"Aktuální cena akcie {ticker.upper()} je {price:.2f} USD."
    except Exception as e:
        return f"Nepodařilo se získat cenu pro {ticker.upper()}: {e}"


In [ ]:
from langchain.tools import tool

@tool
def get_stock_price_tool(ticker: str) -> str:
    """Vrací aktuální cenu akcie podle zadaného tickeru (např. AAPL, TSLA)."""
    return get_stock_price(ticker)

from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=openai_api_key,
)

agent = initialize_agent(
    tools=[get_stock_price_tool],
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Testování
agent.run("Jaká je aktuální cena akcie AAPL?")



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [30]:
from openai import OpenAI
import requests
import json
client = OpenAI()


def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

context = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="o3",
    input=context,
    tools=tools,
    store=False,
    include=["reasoning.encrypted_content"] # Encrypted chain of thought is passed back in the response
)


context += response.output # Add the response to the context (including the encrypted chain of thought)
tool_call = response.output[1]
args = json.loads(tool_call.arguments)



result = get_weather(args["latitude"], args["longitude"])

context.append({                               
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="o3",
    input=context,
    tools=tools,
    store=False,
    include=["reasoning.encrypted_content"]
)

print(response_2.output_text)

NotFoundError: Error code: 404 - {'error': {'message': 'Your organization must be verified to use the model `o3-2025-04-16`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [39]:
import openai

models = openai.models.list()
for model in models.data:
    print(model.id)


text-embedding-ada-002
whisper-1
gpt-3.5-turbo
tts-1
gpt-3.5-turbo-16k
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
o1-preview-2024-09-12
o1-preview
o1-mini-2024-09-12
o1-mini
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-audio-preview
gpt-4o-2024-11-20
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-image-1
